In [4]:
import os
from flask import Flask, request, jsonify
from tensorflow.keras import models
from PIL import Image
import numpy as np
import threading

# Load the model
model = models.load_model("baseline.keras")

# Class names for CIFAR-10 dataset
class_names = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck',
}

app = Flask(__name__)

# Function to process the image and predict
def predict_image(model, img):
    img = img.convert("RGB")
    img = img.resize((32, 32))
    data = np.asarray(img)
    data = data / 255.0
    probs = model.predict(np.array([data])[:1])

    top_prob = probs.max()
    top_pred = class_names[np.argmax(probs)]
    
    return top_prob, top_pred

# Route for prediction
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({"error": "No file part"}), 400

    file = request.files['file']

    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    try:
        # Open and process the image
        image = Image.open(file)
        prob, pred = predict_image(model, image)

        # Return the prediction and probability
        return jsonify({
            "prediction": pred,
            "probability": round(prob * 100, 2)
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run the app in a background thread
def run_app():
    app.run(host="0.0.0.0", port=5005)

# Start the Flask app in a separate thread
if __name__ == "__main__":
    thread = threading.Thread(target=run_app, daemon=True)
    thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5005
 * Running on http://192.168.31.254:5005
Press CTRL+C to quit
192.168.31.254 - - [10/Sep/2024 19:13:28] "GET / HTTP/1.1" 404 -
192.168.31.254 - - [10/Sep/2024 19:13:28] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Sep/2024 19:13:34] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [10/Sep/2024 19:13:34] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Sep/2024 19:13:39] "GET / HTTP/1.1" 404 -
192.168.31.254 - - [10/Sep/2024 19:13:49] "GET / HTTP/1.1" 404 -
192.168.31.254 - - [10/Sep/2024 19:19:04] "GET / HTTP/1.1" 404 -
192.168.31.254 - - [10/Sep/2024 19:19:36] "POST / HTTP/1.1" 404 -
